In [ ]:
# Import python packages
import modin.pandas as pd
import snowflake.snowpark.modin.plugin

import plotly.express as px

from sklearn.preprocessing import MinMaxScaler

# Set snowpark session
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
SELECT 
    t.geo_name as country,
    t.date,
    t.value AS trade_bal,
    r.value AS reserve_bal,
    gg.value AS gdp_growth, -- as % QoQ
    gr.value AS debt_gdp_ratio,
    i.real_interest_rate
FROM FX_INVESTMENT.MACROECON.TRADE_BALANCE t
JOIN FX_INVESTMENT.MACROECON.RESERVES r
ON t.geo_name = r.GEO_NAME and t.date = r.date
JOIN FX_INVESTMENT.MACROECON.GDP_GROWTH gg 
on t.geo_name = gg.geo_name and t.date = gg.date
JOIN FX_INVESTMENT.MACROECON.GDP_RATIOS gr
ON t.geo_name = gr.geo_name AND t.date = gr.date
JOIN FX_INVESTMENT.MACROECON.REAL_INTEREST_RATES i 
ON t.geo_name = i.country AND t.date = i.date
WHERE gr.indicator = 'debt_to_gdp_ratio'
ORDER BY country, date
;

In [ ]:
indicators_df = indicators.to_pandas()
indicator_cols = ['TRADE_BAL', 'RESERVE_BAL', 'GDP_GROWTH', 'DEBT_GDP_RATIO', 'REAL_INTEREST_RATE']
indicators_df.head()

In [ ]:
indicators_df.describe()

In [ ]:
# normalize indicator columns using MinMax scaler
scaler = MinMaxScaler()
scaled_indicators = scaler.fit_transform(indicators_df[indicator_cols])

indicators_df[indicator_cols] = scaled_indicators
indicators_df.head()

In [ ]:
indicators_df['DEBT_SCORE'] = 1 - indicators_df['DEBT_GDP_RATIO'] # a lower debt-to-gdp is better
indicator_cols.append('DEBT_SCORE')
indicator_cols.remove('DEBT_GDP_RATIO')

indicators_df['FX_COMPOSITE_SCORE'] = indicators_df[indicator_cols].sum(axis=1)

In [ ]:
indicators_df['FX_COMPOSITE_SCORE'].describe()

In [ ]:
indicators_df = indicators_df.sort_values(by = ['COUNTRY', 'DATE'])

In [ ]:
fig = px.line(indicators_df,
             x='DATE',
             y='FX_COMPOSITE_SCORE',
             color='COUNTRY')

# Generate dropdown options dynamically
countries = indicators_df['COUNTRY'].unique()
buttons = [{"label": "All Countries", "method": "update", "args": [{"visible": [True] * len(countries)}, {"title": "All Countries"}]}]

for i, country in enumerate(countries):
    visibility = [j == i for j in range(len(countries))]
    buttons.append({"label": country, "method": "update", "args": [{"visible": visibility}, {"title": country}]})

# Add dropdown to the figure
fig.update_layout(
    updatemenus=[
        {
            "buttons": buttons,
            "direction": "down",
        }
    ]
)

In [ ]:
# get the latest available score
indicators_df.groupby("COUNTRY") \
    .last()[['DATE', 'FX_COMPOSITE_SCORE']] \
    .sort_values(by='FX_COMPOSITE_SCORE', ascending = False)

While the data is only as recent as the end of 2023, South Korea has the strongest FX Score which aligns with my initial submission.

If my method is flawed, it's at least consistent. 

However, I would probably recommend investing in GBP or CAD as their scores are trending positvely upward but not peaked like the KRW. This makes me think there is more room for appreciation of the currencies.